## Setup PyDrake in Google Colab

In [4]:
import sys
import importlib
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200918'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)
  !pip install pyngrok==4.2.2

## Import Libraries

In [7]:
# Do general python imports

import os
import numpy as np
import matplotlib.pyplot as plt

# Drake Imports

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, MultibodyPlant, Variable, Simulator)
from pydrake.all import Linearize, LinearQuadraticRegulator
from pydrake.all import plot_system_graphviz, PlanarSceneGraphVisualizer

## Setup URDF Model Path

In [9]:
if 'google.colab' in sys.modules:
  # Check if this cell has already been run:
  if not os.path.isdir('DrakeTests/acrobot_vs_pendulum/'):
    !git clone https://github.com/vyas-shubham/DrakeTests.git
  urdfFolderPath = 'DrakeTests/acrobot_vs_pendulum/'
else:
  urdfFolderPath = ''

# Comment/Uncomment the lines below to select a S/C URDF
filePath = urdfFolderPath+'Acrobot.urdf'
assert os.path.isfile(filePath)

## Setup Diagram and Run Simulation

The error is at the Linearization/LQR line even though the error is about visualization.

```
RuntimeError: The geometry query input port (see MultibodyPlant::get_geometry_query_input_port()) of this MultibodyPlant is not connected. Please connect thegeometry query output port of a SceneGraph object (see SceneGraph::get_query_output_port()) to this plants input port in a Diagram.
```

In [10]:
duration = 10.0
# Set up a block diagram with the robot (dynamics) and a visualization block.
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0001)

# Load the Acrobot description from URDF file
parser = Parser(plant, scene_graph)
parser.AddModelFromFile(filePath)

# # Reduce/remove gravity
# plantGravityField = plant.gravity_field()
# plantGravityField.set_gravity_vector([0,0,0])

plant.Finalize()

# Linearize the MBP about [pi, 0, 0, 0]
q = [np.pi, 0]
q_dot = [0, 0]

linearize_context = plant.CreateDefaultContext()
plant.SetPositions(linearize_context, q)
plant.SetVelocities(linearize_context, q_dot)
actuation_port = plant.get_actuation_input_port()
actuation_port.FixValue(linearize_context, [0])

# Set up LQR, with high velocity gains to try to ensure that the end state is zero-velocity
# without caring much for the final orientation.
Q = np.diag([0.001, 0.001, 10, 10])
# For testing, high position gains as well.
# Q = np.diag([100, 100, 100, 100])
R = np.identity(1)
lqr_controller = builder.AddSystem(LinearQuadraticRegulator(plant, linearize_context, Q, R,
                                                            input_port_index=actuation_port.get_index()))

# Connect the LQR Controller to MBP
builder.Connect(plant.get_state_output_port(), lqr_controller.get_input_port(0))
builder.Connect(lqr_controller.get_output_port(0), plant.get_actuation_input_port())

# Add a Planar Visualization instead of a 3-D Meshcat Visualization
visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, 
                          xlim=[-4., 4.], ylim=[-4., 4.], show=True))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
            visualizer.get_input_port(0))


# Set up a simulator to run the
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyMutableContextFromRoot(context)
actuation_port.FixValue(plant_context, [0])

for i in range(1):
    context.SetTime(0.)
    randState =  np.array(q+q_dot) + 0.3 * np.random.randn(4,)
    randState[-2:0] = 4 * randState[-2:0]
    print(randState)
    plant_context.SetDiscreteState(randState)
#     plant_context.SetContinuousState(randState)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    simulator.AdvanceTo(duration)


RuntimeError: The geometry query input port (see MultibodyPlant::get_geometry_query_input_port()) of this MultibodyPlant is not connected. Please connect thegeometry query output port of a SceneGraph object (see SceneGraph::get_query_output_port()) to this plants input port in a Diagram.